In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 90)

In [6]:
in1 = pd.read_csv('./RESEARCH_TUMOR.dsv', delimiter='\t', low_memory=False)
in2 = in1[in1['ORGAN']=="colon"]
print in2.columns

Index([u'CASE_NO', u'ACCRUAL_ID', u'TUMOR_ID', u'ORGAN', u'LOCALIZATION', u'LOCALIZATION_VALUE', u'LOCALIZATION_DETAIL', u'CLASSIFICATION_ORGAN', u'HISTOLOGIC_TYPE', u'MORPHOLOGY_ICDO_CODE', u'MORPHOLOGY_ICDO_DESC', u'PTNM_EDITION', u'PTNM_T', u'PTNM_M', u'PTNM_N', u'PTNM_N_POSITIVE', u'PTNM_N_TOTAL', u'PTNM_L', u'PTNM_V', u'PRADICALITY', u'STAGE', u'RADICALITY', u'GRADING', u'MALIGNANCE', u'CLASSIFICATION_COMMENT', u'TOTAL_LENGTH', u'TUMOR_SIZE', u'DISTANCE_TO_BORDER', u'RESECTION_PROCEDURE', u'RESECTION_TYPE', u'TRANSECTION_START', u'TRANSECTION_END', u'TRANSSECTION_DISTAL_TIME', u'TRANSSECTION_PROXIMAL_TIME', u'RESECTION_COMPLETED_TIME', u'PRESENTATION_TO_SURGEON_TIME', u'SAMPLES_FREEZING_TIME', u'SAMPLES_FORMALIN_TIME', u'SAMPLES_GLUTARALDEHYDE_TIME', u'VESSEL_RESECTION_COMPLET_TIME', u'VESSEL_SAMPLES_FREEZING_TIME', u'VESSEL_SAMPLES_FORMALIN_TIME', u'RESECTION_COMPL_TO_FREEZE', u'RESECTION_COMPL_TO_FORMALIN', u'VESSEL_RESEC_COMPL_TO_FREEZE', u'VESSEL_RESEC_COMPL_TO_FORMALIN', u'TR

In [5]:
print in1.shape, in2.shape

(25722, 82) (6253, 82)


In [7]:
tcga_case = pd.read_csv('./tcga_caseID_171.lst', delim_whitespace=True)
tcga_case['CASE_NO'] = tcga_case.CaseID.map(lambda x: x.split('-')[0])
print (tcga_case.CASE_NO.size)
tcga_case.head()

171


,TCGA,CaseID,CASE_NO
0,TCGA-AA-3526,A1888-Tp2,A1888
1,TCGA-AA-3831,B1164-Tp1,B1164
2,TCGA-AA-3529,A1862-Tp1,A1862
3,TCGA-AA-A01C,A2733,A2733
4,TCGA-AA-3549,A2327-Tp1,A2327


In [9]:
case_171  = pd.merge(tcga_case, in1, how='inner')

In [10]:
print "Original case_171 SHAPE:",case_171.shape
print "\nDrop variables with >20% NaN values \n"
case_171 = case_171.dropna(axis=1, thresh = 0.8*len(case_171))
print "case_171 SHAPE:",case_171.shape

Original case_171 SHAPE: (195, 84)

Drop variables with >20% NaN values 

case_171 SHAPE: (195, 48)


In [7]:
## Select columns with biological relevance to our study 
case_171 = case_171[['CASE_NO','TCGA','LOCALIZATION','CLASSIFICATION_ORGAN','PRADICALITY','STAGE',
                     'RADICALITY','GRADING','TOTAL_LENGTH','TUMOR_SIZE']]
print "\nNaN distribution : \n"
print (case_171.isnull().sum(axis=0)*100/case_171.shape[0])


NaN distribution : 

CASE_NO                 0.000000
TCGA                    0.000000
LOCALIZATION            0.000000
CLASSIFICATION_ORGAN    0.000000
PRADICALITY             7.179487
STAGE                   5.641026
RADICALITY              8.717949
GRADING                 5.128205
TOTAL_LENGTH            1.025641
TUMOR_SIZE              1.025641
dtype: float64


In [9]:
print "##LOCALIZATION##\n"
print case_171.LOCALIZATION.value_counts()/case_171.shape[0] * 100
print "\n####STAGE####\n"
print case_171.STAGE.value_counts()/case_171.shape[0] * 100
print "\n####GRADING####\n"
print case_171.GRADING.value_counts()/case_171.shape[0] * 100
print "\n####PRADICALITY####\n"
print case_171.PRADICALITY.value_counts()/case_171.shape[0] * 100

##LOCALIZATION##

sigmoid colon       40.512821
ascending colon     20.000000
ileocaecal          16.923077
transverse colon     8.717949
right flexure        5.641026
descending colon     4.102564
rectum               2.564103
left flexure         1.538462
Name: LOCALIZATION, dtype: float64

####STAGE####

IIA      23.076923
I        18.461538
IV       13.333333
II       12.820513
III       9.230769
III C     7.179487
III B     5.128205
III A     2.564103
IIB       2.051282
IVA       0.512821
Name: STAGE, dtype: float64

####GRADING####

G2    64.615385
G3    29.743590
G1     0.512821
Name: GRADING, dtype: float64

####PRADICALITY####

R0    91.282051
R1     1.538462
Name: PRADICALITY, dtype: float64


In [11]:
#We drop all columns in which one bin has >80% distribution
case_171 = case_171[['TCGA','CASE_NO','STAGE','GRADING','LOCALIZATION']]

case_171['STAGE'].replace(['IIA','IIB'],'II', inplace=True)
case_171['STAGE'].replace(['III A','III B','III C'],'III', inplace=True)
case_171['STAGE'].replace(['IVA','IVB'],'IV', inplace=True)

print "## New STAGE distribution ## \n"
print case_171.STAGE.value_counts()/case_171.shape[0] * 100

case_171['LOCALIZATION'].replace(['right flexure','left flexure'],'flexture',inplace=True)
case_171['LOCALIZATION'].replace(['descending colon','rectum'],'descending',inplace=True)
print "##New LOCALIZATION distribution##\n"
print ( case_171.LOCALIZATION.value_counts() /case_171.shape[0] * 100)

# NOTE :  There are more than one instance favailable for a few cases where tumor is found in multiple location 
# and the grade of tumor can be different for different locations
case_171.head(20)

## New STAGE distribution ## 

II     37.948718
III    24.102564
I      18.461538
IV     13.846154
Name: STAGE, dtype: float64
##New LOCALIZATION distribution##

sigmoid colon       40.512821
ascending colon     20.000000
ileocaecal          16.923077
transverse colon     8.717949
flexture             7.179487
descending           6.666667
Name: LOCALIZATION, dtype: float64


,TCGA,CASE_NO,STAGE,GRADING,LOCALIZATION
0,TCGA-AA-3526,A1888,I,G2,sigmoid colon
1,TCGA-AA-3831,B1164,II,G3,sigmoid colon
2,TCGA-AA-3529,A1862,III,G2,sigmoid colon
3,TCGA-AA-3529,A1862,III,G2,descending
4,TCGA-AA-A01C,A2733,III,G2,transverse colon
5,TCGA-AA-A01C,A2733,III,G2,ascending colon
6,TCGA-AA-3549,A2327,I,G2,ileocaecal
7,TCGA-AA-3696,A2779,IV,G2,sigmoid colon
8,TCGA-AA-A03F,A607,III,G2,ileocaecal
9,TCGA-AA-3697,A302,II,G2,sigmoid colon


In [12]:
cols = ['STAGE','GRADING','LOCALIZATION']
S_series = case_171.groupby('CASE_NO')['STAGE'].apply(list)
G_series = case_171.groupby('CASE_NO')['GRADING'].apply(list)
L_series = case_171.groupby('CASE_NO')['LOCALIZATION'].apply(list)

S_s1 = pd.Series()
G_s1 = pd.Series()
L_s1 = pd.Series()

for ix, val in S_series.iteritems() :
    S_series[ix] = list(set(S_series[ix]))
    if (np.nan in S_series[ix]): 
        S_series[ix].remove(np.nan) 
    if len(S_series[ix]) > 1:       
        S_s1[ix] = S_series[ix][-1]
    elif len(S_series[ix]) == 1: 
        S_s1[ix] = S_series[ix][0]
    elif len(S_series[ix]) == 0: 
        S_s1[ix] = np.nan
        
for ix, val in L_series.iteritems() :
    L_series[ix] = list(set(L_series[ix]))
    if (np.nan in L_series[ix]): 
        L_series[ix].remove(np.nan)  
    if len(L_series[ix]) > 1:  
        L_s1[ix] = 'multiple'
    elif len(L_series[ix]) == 1: 
        L_s1[ix] = L_series[ix][-1]
    elif len(L_series[ix]) == 0: 
        L_s1[ix] = np.nan
        
for ix, val in G_series.iteritems() :
    G_series[ix] = list(set(G_series[ix]))
    if (np.nan in G_series[ix]): 
        G_series[ix].remove(np.nan)  
    if len(G_series[ix]) > 1:  
        G_s1[ix] = G_series[ix][0]
    elif len(G_series[ix]) == 1:
        G_s1[ix] = G_series[ix][0]
    elif len(G_series[ix]) == 0:
        G_s1[ix] = np.nan

In [23]:
#list(set(['G3', 'G2','G4']))
print "#### Before cleaning data ####\n"
print L_series.head(10)
print "\n#### After cleaning ####\n"
print L_s1.head(10)

#### Before cleaning data ####

CASE_NO
A1023           [sigmoid colon]
A1064           [sigmoid colon]
A1101           [sigmoid colon]
A114            [sigmoid colon]
A1143              [ileocaecal]
A117     [descending, flexture]
A1193              [ileocaecal]
A121          [ascending colon]
A1211              [ileocaecal]
A1230              [ileocaecal]
Name: LOCALIZATION, dtype: object

#### After cleaning ####

A1023      sigmoid colon
A1064      sigmoid colon
A1101      sigmoid colon
A114       sigmoid colon
A1143         ileocaecal
A117            multiple
A1193         ileocaecal
A121     ascending colon
A1211         ileocaecal
A1230         ileocaecal
dtype: object


In [13]:
# Merge series into one frame 
frames= [G_s1,L_s1,S_s1]
case_171_merged = pd.concat([G_s1,L_s1,S_s1], axis =1)
case_171_merged.reset_index(inplace=True)
case_171_merged.columns = ['CASE_NO','GRADE','LOCALIZATION','STAGE']
case_171_merged.shape

(171, 4)

In [14]:
case_171_merged.head()

,CASE_NO,GRADE,LOCALIZATION,STAGE
0,A1023,G2,sigmoid colon,II
1,A1064,G2,sigmoid colon,II
2,A1101,G2,sigmoid colon,IV
3,A114,G3,sigmoid colon,IV
4,A1143,G3,ileocaecal,II


In [206]:
print (case_171_merged.isnull().sum(axis=0)*100/case_171_merged.shape[0])

CASE_NO         0.000000
GRADE           0.000000
LOCALIZATION    0.000000
STAGE           0.584795
dtype: float64


In [15]:
## Find median of a column and then replace NaN with median , NOTE: not the most efficient way to fill NaN

#STAGE_median = case_171.STAGE.value_counts().reset_index().sort_values(by=['STAGE','index'],ascending=[False,False]).loc[0]['index']
#case_171['STAGE'].fillna(STAGE_median, inplace=True)

#GRADING_median = case_171.GRADING.value_counts().reset_index().sort_values(by=['GRADING','index'],ascending=[False,False]).loc[0]['index']
#case_171['GRADING'].fillna(GRADING_median, inplace=True)

### A **better** way to fill NaN in every column with its own most frequent value for the entire dataframe

case_171_merged = case_171_merged.apply(lambda x:x.fillna(x.value_counts().index[0]))

print (case_171_merged.isnull().sum(axis=0)*100/case_171_merged.shape[0])

CASE_NO         0.0
GRADE           0.0
LOCALIZATION    0.0
STAGE           0.0
dtype: float64


In [16]:
print ( case_171_merged.LOCALIZATION.value_counts() /case_171_merged.shape[0] * 100)

sigmoid colon       40.935673
ascending colon     18.713450
ileocaecal          16.374269
transverse colon     7.602339
multiple             6.432749
flexture             5.847953
descending           4.093567
Name: LOCALIZATION, dtype: float64


In [17]:
## Assign numeric labels to categorical values for all the colums using "LabelEncoder"

cols = ['GRADE','LOCALIZATION','STAGE']
RTR_3cv_171cases = pd.DataFrame()
RTR_3cv_171cases['CASE_NO']=case_171_merged['CASE_NO']
for x in cols:
    number = LabelEncoder()
    RTR_3cv_171cases[x] = number.fit_transform(case_171_merged[x].astype('str'))
    print number.classes_    

['G1' 'G2' 'G3']
['ascending colon' 'descending' 'flexture' 'ileocaecal' 'multiple'
 'sigmoid colon' 'transverse colon']
['I' 'II' 'III' 'IV']


In [217]:
RTR_3cv_171cases.to_csv('./COAD_clinical_matrix/RTR_3cv_171cases.csv', sep = '\t', index=False)
RTR_3cv_171cases.head(10)

,CASE_NO,GRADE,LOCALIZATION,STAGE
0,A1023,1,5,1
1,A1064,1,5,1
2,A1101,1,5,3
3,A114,2,5,3
4,A1143,2,3,1
5,A117,1,4,0
6,A1193,1,3,1
7,A121,1,0,1
8,A1211,1,3,2
9,A1230,1,3,1
